In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
import random

with open('questions.md', 'r') as file:
    query_list = file.read().splitlines()

CONFIG = {
    'csv_path': './data/text_chunks_with_embeddings.csv',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'attn_implementation': 'sdpa',
    'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct',
    'embedding_model_id': 'all-mpnet-base-v2',
}

SYS_PROMPT = {
    "education": """
    You are Study-Buddy. An educational chatbot that will aid students in their studies.
    You are given the extracted parts of curriculum specific documents and a question. Provide a conversational and educational answer with good and easily read formatting.
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    If you don't know the answer, just say "I do not know." Don't make up an answer.
    """
}

def import_chunks_with_embeddings(csv_path: str):
    """
    Imports the chunks with embeddings from a csv file.
    """
    text_chunks_with_embeddings_df = pd.read_csv(csv_path, index_col=0)
    text_chunks_with_embeddings_df['embedding'] = text_chunks_with_embeddings_df['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    chunks_with_embeddings = text_chunks_with_embeddings_df.to_dict(orient='records')
    return chunks_with_embeddings

def get_chunks_embeddings_as_tensor(chunks_with_embeddings: list[dict]):
    """
    Converts the embeddings of chunks to a tensor.
    """
    embeddings_list = [chunk['embedding'] for chunk in chunks_with_embeddings]
    embeddings = torch.tensor(np.stack(embeddings_list, axis=0), dtype=torch.float32)
    # embeddings = torch.tensor(np.stack(chunks_with_embeddings['embedding'].tolist(), axis=0), dtype=torch.float32)
    return embeddings



def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                embedding_model: SentenceTransformer,
                                n_resources_to_return: int=5):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """
    # Embed query
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    
    # Get dot product scores on embeddings
    dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
    
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices


def generate_model_response(prompt: str, tokenizer, model, terminators, device="cuda"):
    input_ids = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids, 
        max_new_tokens=1024, 
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response)


def get_user_prompt(query: str, retrieved_documents: list[dict]):
    """
    Formats the prompt with the query and the retreived documents.
    """
    base_prompt = f"Query: {query}\nContext:"
    for item in retrieved_documents:
        base_prompt += f"\n- {item['text']}"
    return base_prompt

def format_prompt(formatted_prompt: str):
    message = [
        { "role": "system", "content": SYS_PROMPT['education'] },
        { "role": "user", "content": formatted_prompt }
    ]
    return message
    
def generate_response(query: str, embeddings: torch.tensor, embedding_model: SentenceTransformer, tokenizer, model, terminators, chunks_with_embeddings: list[dict]):
    scores, indices = retrieve_relevant_resources(
        query=query, 
        embeddings=embeddings, 
        embedding_model=embedding_model)
    user_prompt = get_user_prompt(query=query, retrieved_documents=[chunks_with_embeddings[i] for i in indices])
    formatted_prompt = format_prompt(user_prompt) 
    return generate_model_response(formatted_prompt, tokenizer, model, terminators)

In [2]:
chunks_with_embeddings = import_chunks_with_embeddings(CONFIG['csv_path'])
embeddings = get_chunks_embeddings_as_tensor(chunks_with_embeddings).to(CONFIG['device'])

embedding_model = SentenceTransformer(model_name_or_path=CONFIG['embedding_model_id'], device=CONFIG['device'])
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=CONFIG['model_id'])
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=CONFIG['model_id'], 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=False, 
    attn_implementation=CONFIG['attn_implementation']
    ).to(CONFIG['device'])

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

FileNotFoundError: [Errno 2] No such file or directory: '../data/text_chunks_with_embeddings.csv'

In [ ]:
query = random.choice(query_list)
response = generate_response(query)
print(f"Query: {query}")
print(response)